In [1]:
import pandas as pd
import numpy as np
#https://machinelearningmastery.com/divergence-between-probability-distributions/
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import sqlalchemy

df_stikhi = pd.read_csv('./101120_stikhicats.csv', index_col = 0)
df_manifest = pd.read_csv('./083020_manifestcats.csv', index_col = 0)
df_dnevnik = pd.read_csv('./072420_dnevnikcats.csv', index_col = 0)
df_roman = pd.read_csv('./101820_romancats.csv', index_col = 0)

In [2]:
#start sql

passw = 'aik.9Yor'
engine = sqlalchemy.create_engine('mysql+pymysql://madkehl:' + passw + '@127.0.0.1/prozhito_orig', encoding = 'utf-8')
connection = engine.connect()
metadata = sqlalchemy.MetaData()
notes = sqlalchemy.Table('notes', metadata, autoload = True, autoload_with = engine)
persons = sqlalchemy.Table('persons', metadata, autoload = True, autoload_with = engine)
diary =sqlalchemy.Table('diary', metadata, autoload = True, autoload_with = engine)

def execute(query):
    ResultProxy = connection.execute(query)
    return ResultProxy


def countyear(list_years):
    '''
    SELECT COUNT(YEAR(date)) AS NEWYEAR from notes WHERE YEAR(date) in (1901, 1903, 1906) GROUP BY YEAR(date); 
    **for some reason the year column is less complete than date column
    '''
    trial = execute('SELECT YEAR(date), COUNT(YEAR(date))FROM notes WHERE YEAR(date) in ('+ (list_years) + ') GROUP BY YEAR(date);')
    ResultSet = trial.fetchall()
    return ResultSet


C:\Users\madke\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning:

(3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")



In [15]:
def cats2scatter(df_cats, title):
    '''
    INPUT:  df with categories, title of genre (str)
    
    OUTPUT: plotly scatter object with counts over years
    '''
    df_cats_wide = pd.get_dummies(df_cats['entry_type'])
    df_cats_wide['entry_year'] = df_cats['entry_year']
    df_cats_wide = df_cats_wide[df_cats_wide['entry_year'] < 2000]
    df_cats_yearsums = df_cats_wide.groupby('entry_year').agg(sum)
    df_graph = df_cats_yearsums.sum(axis = 1)
    plot = go.Scatter(x = df_graph.index, y = df_graph.values, name = title)
    return(plot)

def cats2scatter_normed(df_cats, title):
    '''
    INPUT:  df with categories, title of genre (str)
    
    OUTPUT: plotly scatter object with counts over years
    '''
    df_cats_wide = pd.get_dummies(df_cats['entry_type'])
    df_cats_wide['entry_year'] = df_cats['entry_year']
    df_cats_wide = df_cats_wide[df_cats_wide['entry_year'] < 2000]
    
    df_cats_wide =df_cats_wide.sort_values(by = 'entry_year', axis = 0).reset_index(drop = True)
    
    df_cats_yearsums = df_cats_wide.groupby('entry_year').agg(sum)
    df_graph = pd.DataFrame(df_cats_yearsums.sum(axis = 1))
    
    input_ls = [str(i) for i in list(df_graph.index)]
    input_str = ', '.join(input_ls)
    
    new_year = countyear(input_str)
    year_sorted = sorted(new_year, key= lambda x: x[0])
    year_totals = [i[1] for i in year_sorted]
    
    df_graph['normalized'] = df_graph[0]/year_totals
    
    plot = go.Scatter(x = df_graph.index, y = (df_graph['normalized']), name = title)
    return(plot)


In [20]:
fig = make_subplots(rows=2, cols=2,
          subplot_titles=("Poetry","Novels", "Manifesto", "Diaries"))

fig.add_trace(cats2scatter_normed(df_stikhi, 'Poetry, % of Entries'), row=1, col=1)
fig.add_trace(cats2scatter_normed(df_roman, 'Novels, % of Entries'), row=1, col=2)
fig.add_trace(cats2scatter_normed(df_manifest, 'Manifesto/Essay, % of Entries'), row=2, col=1)
fig.add_trace(cats2scatter_normed(df_dnevnik, 'Diaries, % of Entries'), row=2, col=2)

#fig.update_yaxes(type="log")

fig.update_layout(title_text="% entries by year mention",
                  title_font_size=20)


In [29]:
fig = go.Figure()


fig.add_trace(cats2scatter(df_stikhi, 'Poetry'))
fig.add_trace(cats2scatter(df_roman, 'Novels'))
fig.add_trace(cats2scatter(df_manifest, 'Manifesto/Essay'))
fig.add_trace(cats2scatter(df_dnevnik, 'Diaries'))


In [32]:
fig = make_subplots(rows=2, cols=2,
          subplot_titles=("Poetry","Novels", "Manifesto", "Diaries"))

fig.add_trace(cats2scatter(df_stikhi, 'Poetry'), row=1, col=1)
fig.add_trace(cats2scatter(df_roman, 'Novels'), row=1, col=2)
fig.add_trace(cats2scatter(df_manifest, 'Manifesto/Essay'), row=2, col=1)
fig.add_trace(cats2scatter(df_dnevnik, 'Diaries'), row=2, col=2)